Through some bugs in 00_convert_seisandb_to_csvfiles.py, I ended up with many duplicate lines in the files like reawav_MVOE_YYYYMM.csv. 

This code sorts that out, and the new files are like REA_WAV_MVOE_YYYY_MM.csv

It also sorts out some problems with the Seisan database, e.g. 1-many mapping from WAV files to S-files

Glenn Thompson, 2021/10/27 on hal

In [2]:
# Remove duplicates from reawav_MVOE_YYYYMM.csv files
import pandas as pd
import os, glob
SEISAN_DATA = os.path.join( os.getenv('HOME'),'DATA','MVO')
SEISAN_DB = 'MVOE_'
csvfiles = sorted(glob.glob(os.path.join(SEISAN_DATA,'reawav_' + SEISAN_DB + '*.csv')))
frames = []
for csvfile in csvfiles:
    df = pd.read_csv(csvfile)
    frames.append(df) 
dfall = pd.concat(frames, sort=True)
dfall.drop_duplicates(inplace=True)
#dfall.set_index('filetime', inplace=True) # we will need this later to remerge
#dfall.sort_index(inplace=True)
allcsv = os.path.join(SEISAN_DATA, 'reawav_%sall.csv' % SEISAN_DB)


In [5]:
SEISAN_TOP = os.getenv('SEISAN_TOP')
SEISAN_TOP = '/media/sdd1/backups/seismo' # Uncomment to use local archive on hal, rather than newton

print('Length of DataFrame is %d ' % len(dfall))

# with vi, i see that there are many almost duplicate rows except for rounding errors
# let's drop duplicates based on just sfile and path (path is wavfile)
reawavdf2 = dfall.drop_duplicates(
    subset=['sfile','path'],
    keep = 'last')
print('Length after drop_duplicates is %d ' % len(reawavdf2))

# Only rows left now will have unique sfile-path combinations

# Now we want to deal with events that have multiple S-files, mainly because there are DSN (MVO) and ASN (SPN) WAV files,
# with different start times
# Find rows of reawavdf2 with a non-unique (WAVfile) path
duplicate_rows = reawavdf2[reawavdf2.duplicated(subset=['path'])]
incorrect_sfiles = []
for i,thispath in enumerate(duplicate_rows['path']):
    sameWAVdf = reawavdf2[reawavdf2['path']==thispath]

    # Find correct Sfile
    WAV_DHMS = []
    print('\n\nMatch %d' %i)
    for i2,row in sameWAVdf.iterrows():
        print('SFILE=',row['sfile'])
        wavpath = os.path.join(SEISAN_TOP, row['path'])
        readir = os.path.join(SEISAN_TOP, os.path.dirname(row['path']).replace('WAV','REA'))
        sfilepath = os.path.join(readir, row['sfile'])
        with open(sfilepath) as f:
            lines = f.readlines()
            for line in lines:
                line = line.strip()
                if len(line)>0:
                    if line[-1]=='6':
                        if line[0]=='2' or line[0]=='1':
                            WAV_DHMS.append(line[8:18])
                        elif line[0]=='9':
                            WAV_DHMS.append(line[5:15])
    WAV_DHMS=sorted(WAV_DHMS)
    print('WAV_DHMS = %s' % WAV_DHMS[0])
    for i3,row in sameWAVdf.iterrows():    
        if WAV_DHMS[0] in row['sfile'] and not '-temp' in row['sfile']:
            print('Correct SFILE is %s' % row['sfile'])
        else:
            incorrect_sfiles.append(row['sfile'])
    
# Remove rows matching incorrect_sfiles from reawavdf2  
dfall = reawavdf2[~reawavdf2['sfile'].isin(incorrect_sfiles)]
print('Length after dropping bad sfiles is %d ' % len(dfall))
dfall.to_csv(allcsv)

Length of DataFrame is 208189 
Length after drop_duplicates is 208189 


Match 0
SFILE= 08-2136-38L.S200604-temp
SFILE= 08-2136-38L.S200604
WAV_DHMS = 08-2136-38
Correct SFILE is 08-2136-38L.S200604


Match 1
SFILE= 29-1227-26L.S200604-temp
SFILE= 29-1227-26L.S200604
WAV_DHMS = 29-1227-26
Correct SFILE is 29-1227-26L.S200604
Length after dropping bad sfiles is 208187 


In [10]:
years = list(set(sorted(dfall.year)))
for thisyear in years:
    dfyear = dfall[dfall.year == thisyear]
    months = list(set(sorted(dfyear.month)))
    print(thisyear, months)
    for thismonth in months:
        dfyearmonth = dfyear[dfyear.month == thismonth]
        yearmonthcsv = os.path.join(SEISAN_DATA, 'reawav_%s%4d%02d.csv' % (SEISAN_DB, thisyear, thismonth))
        dfyearmonth.to_csv(yearmonthcsv)        

1996 [10, 11, 12]
1997 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
1901 [3]
1998 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
1999 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
2000 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
2001 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
2002 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
2003 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
2004 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
2005 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
2006 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
2007 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
2008 [1, 2, 3, 4, 5, 6, 7, 8]
